# Checksums

In computing, we often need to maintain the integrity of transmitted data. This requires a way to check that the source data and the received data are in fact the same, without having to compare every single bit.

One way to do so is to calculate a **checksum** for each block of data.

## Example 1: Luhn algorithm (mod 10)

Main article: [Luhn algorithm (Wikipedia)](https://en.wikipedia.org/wiki/Luhn_algorithm)

The Luhn algorithm is named after its creator, Hans Peter Luhn, a scientist at IBM. It is used to verify ID numbers all over the world.

The formula verifies a number against its included check digit, and its instructions are as follows:

1. From the rightmost digit (excluding the check digit) and moving left, double the value of every second digit. The check digit is neither doubled nor included in this calculation; the first digit doubled is the digit located immediately left of the check digit. If the result of this doubling operation is greater than 9 (e.g., 8 × 2 = 16), then add the digits of the result (e.g., 16: 1 + 6 = 7, 18: 1 + 8 = 9) or, equivalently, subtract 9 from the result (e.g., 16: 16 − 9 = 7, 18: 18 − 9 = 9).
2. Take the sum of all the digits (including the check digit).
3. If the total modulo 10 is equal to 0 (if the total ends in zero) then the number is valid according to the Luhn formula; otherwise it is not valid.

### Pseudocode

    function checkLuhn(string purportedCC) {
        int nDigits := length(purportedCC)
        int sum := integer(purportedCC[nDigits-1])
        int parity := nDigits modulus 2
        for i from 0 to nDigits - 2 {
            int digit := integer(purportedCC[i])
            if i modulus 2 = parity
                digit := digit × 2
            if digit > 9
                digit := digit - 9 
            sum := sum + digit
        }
        return (sum modulus 10) = 0
    }

## Task 1.1

Write program code to implement a function that takes in a string of digits (without checksum digit), and returns a checksum digit.

In [ ]:
# Your code here



## Task 1.2

Write program code that takes in a string of digits (with checksum digit), and returns `True` if it is valid, `False` if it is invalid.

In [ ]:
# Your verification code here



### Test cases

The following strings of digits have the check digit as the last digit. The validity of each string of digits is indicated.

Use them to verify your function.

    79927398710: Invalid
    79927398711: Invalid
    79927398712: Invalid
    79927398713: Valid
    79927398714: Invalid
    79927398715: Invalid
    79927398716: Invalid
    79927398717: Invalid
    79927398718: Invalid
    79927398719: Invalid

In [ ]:
# Your test code here



## Example 2: Binary XOR

Main article: [How is TCP & UDP Checksum Calculated?](https://www.slashroot.in/how-is-tcp-and-udp-checksum-calculated)

Binary XOR is a verification method used for binary data (i.e. "raw" data), frequently found in transmission protocols (e.g. UDP, TCP, and even for writing to storage devices such as hard disks). It works on data sets of any size.

The data is first sliced up into blocks of 16 bits (2 bytes) each. This can be represented as an integer between 0 to 255 (inclusive), or two ASCII characters (8 bits each).

For this example, we will use blocks of 8 bits instead of 16 bits for convenience.

Let's calculate the checksum for a test string:

In [ ]:
teststring = "a sample test string"

Recall that a string is a sequence of *values*, which are converted to characters through an encoding table (such as ASCII, or UTF-8).

For calculating checksums, we don't want to deal with the characters, but with the *value* instead. The `str` data type is not suitable for this; the `bytes` data type is more suitable.

Let's convert our test string to `bytes` using the `bytes()` built-in function:

In [ ]:
teststring_bytes = bytes(teststring, encoding="ascii")
teststring_bytes

### `bytes` data type

The `b` in front of the string indicates that this is a `bytes` object, not a `str`. We can iterate through `bytes` the same way we do with `str`:

In [ ]:
for byte in teststring_bytes:
    print(byte)

This gives us the value of each byte, rather than the ASCII/UTF-8 character which it decodes to.

Just like `str`s, `bytes` objects can be indexed and sliced too:

In [ ]:
# Bytes are integer-indexed
teststring_bytes[0]

In [ ]:
# Bytes can be sliced as well
teststring_bytes[0:6]

Bytes are immutable:

In [ ]:
# Byte indexes cannot be reassigned; this will raise an error
teststring_bytes[0] = 100

If you need a mutable sequence of bytes, convert it into a `bytearray`:

In [ ]:
# Use the bytearray() built-in function to convert to bytearray
mutable_teststring_bytes = bytearray(teststring_bytes)
# Reassign the first character (remember to use the value rather than character)
mutable_teststring_bytes[0] = 100
# Notice that the first letter has changed
mutable_teststring_bytes

### Binary XOR

In binary XOR, we look at each 8-bit block as a sequence of bits. Let's split up our test string into 8-bit blocks (1 value each). Next, we add up all of those values. If the result is greater than 8 bits (max value 255), we take the result modulus 255.

In [ ]:
blocksum = 0
for byte in teststring_bytes:
    blocksum += byte
blocksum = blocksum % 255
blocksum

Now we take the **ones complement** of the result.

The **ones complement** is the number that, when added to the result, will give us a binary sequence that is all `1`s.

The result is 161. In binary, that is `10100001`.  
The **ones complement** of 161 is `01011110` (decimal value 94).  
The sum of 161 and its ones complement is `10100001` + `01011110` = `11111111` (decimal value 255)

Notice that this is equivalent to taking the difference between 255 and the result!

In [ ]:
print(f"255 in binary: {255:#08b}")
print(f"161 in binary: {161:#08b}")
print(f"255 - 161 in binary: {255-161:#08b}")

Technically, this is not subtraction, since we are not performing any carry-over, and negative results are not allowed. This is a **bitwise comparison**.

We are comparing each digit of the result (161) and the largest value (255), and determining the ones complement according to these rules:

1. If both digits are `1` or `0`, the ones complement is `0`.
2. If one digit is `1` and the other digit is `0`, the ones complement is `1`.

This operation is called a **binary XOR** (exclusive OR). Python uses the `^` operator to carry out binary XOR:

In [ ]:
print(f"255 ^ 161: {255 ^ 161}")
print(f"161 ^ 255: {161 ^ 255}")

Notice that regardless of the order of operands, the result is the same; this is not the case for subtraction!

Let's complete our calculation of the checksum for our test string.

We have our result from summing up each 8-bit block of our test string (modulus 255), now let's get the ones complement of the result:

In [ ]:
255 ^ blocksum

This is our checksum value, calculated using binary XOR.

## Task 2.1

Write a function that takes in a string and returns a checksum value calculated using binary XOR (on blocks of 8 bits).

In [ ]:
# Your code here

## Task 2.2

Write a function that takes in a string and a checksum value, and verifies if the string and its accompanying checksum are valid.

In [ ]:
# Your verification code here